In [1]:
import jieba
import pandas as pd
X_train = pd.read_csv("poem_train.csv")
X_test = pd.read_csv("poem_test.csv")
y_train = X_train["作者"]
y_test = X_test["作者"]
X_train

,作者,詩名,內容
0,李白,菩薩蠻·平林漠漠煙如織,平林漠漠煙如織，寒山一帶傷心碧。\r\n暝色入高樓，有人樓上愁。玉階空佇立，宿鳥歸飛急。\r...
1,李白,把酒問月,青天有月來幾時，我今停杯一問之：人攀明月不可得，月行卻與人相隨？皎如飛鏡臨丹闕，綠煙滅儘清輝...
2,李白,春思,燕草如碧絲，秦桑低綠枝。當君懷歸日，是妾斷腸時。春風不相識，何事入羅幃。
3,李白,春夜洛城聞笛,誰家玉笛暗飛聲，散入春風滿洛城。此夜曲中聞折柳，何人不起故園情。
4,李白,古風 其十九,西上蓮花山，迢迢見明星。(西上 一作：西嶽)素手把芙蓉，虛步躡太清。霓裳曳廣帶，飄拂升天行。...
...,...,...,...
2726,白居易,彆元九後詠所懷,零落桐葉雨，蕭條槿花風。悠悠早秋意，生此幽閒中。況與故人彆，中懷正無悰。勿雲不相送，心到青門...
2727,白居易,早秋曲江感懷,離離暑雲散，嫋嫋涼風起。池上秋又來，荷花半成子。朱顏易銷歇，白日無窮已。人壽不如山，年光急於...
2728,白居易,東墟晚歇 時退居渭村。,涼風冷露蕭索天，黃蒿紫菊荒涼田。繞塚秋花少顏色，細蟲小蝶飛翻翻。中有騰騰獨行者，手拄漁竿不騎...
2729,白居易,南秦雪,往歲曾為西邑吏，慣從駱口到南秦。\r\n三時雲冷多飛雪，二月山寒少有春。\r\n我思舊事猶惆...


In [2]:
"""
老師教的LabelEncoder版本：
u = X_train["作者"].unique()
name2index = {n:i for i, n in enumerate(u)}
index2name = {i:n for i, n in enumerate(u)}
y_train = y_train.replace(name2index)
y_test = y_train.replace(name2index)
"""

'\n老師教的LabelEncoder版本：\nu = X_train["作者"].unique()\nname2index = {n:i for i, n in enumerate(u)}\nindex2name = {i:n for i, n in enumerate(u)}\ny_train = y_train.replace(name2index)\ny_test = y_train.replace(name2index)\n'

In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [4]:
# temp = X_train.iloc[1, 2]
# strings = " ".join(jieba.cut(temp)))
# print(X_train["內容"])

def poem2split(s):
    s = s.replace("\r", "").replace("\n", "")
    return " ".join(jieba.cut(s))
# print(poem2split(temp))
X_train["內容"] = X_train["內容"].apply(poem2split)
X_test["內容"] = X_test["內容"].apply(poem2split)

Building prefix dict from the default dictionary ...
Loading model from cache R:\TEMP\jieba.cache
Loading model cost 0.939 seconds.
Prefix dict has been built successfully.


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
tranvec = vec.fit_transform(X_train["內容"]) #稀疏矩陣，只保存有數值資料，儉省RAM空間
testvec = vec.transform(X_test["內容"])
# print(tranvec)
# temp = vec.vocabulary_

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha = 1e-5)
clf.fit(tranvec, y_train)

predict = clf.predict(testvec)
print("預測結果:", list(predict))
print("正確結果:", list(y_test))
print("正確率:", clf.score(testvec, y_test))

預測結果: [0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1]
正確結果: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
正確率: 0.8333333333333334


In [6]:
import numpy as np
name_array = [name for name in le.inverse_transform(np.array([0,1,2]))]

#劃出混淆矩陣
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, clf.predict(testvec))
pd.DataFrame(mat,
             columns = [f"{name}(預測)" for name in name_array],
             index = [f"{name}(實際)" for name in name_array]
            )

,李白(預測),杜甫(預測),白居易(預測)
李白(實際),8,1,1
杜甫(實際),0,9,1
白居易(實際),1,1,8


In [7]:
def poem_predict():
    inp = input("請輸入文字:")
    inpsplit = poem2split(inp)
    inpvec = vec.transform([inpsplit])
    return le.inverse_transform(clf.predict(inpvec))[0]

def poem_predict_all():
    inp = input("請輸入文字：")
    inpsplit = poem2split(inp)
    inpvec = vec.transform([inpsplit])
    pre = clf.predict_proba(inpvec)[0]
    result = []
    for name, proba in zip(name_array, pre):
        result.append(f"{name}機率： {proba:.2f}")
    return result

In [8]:
if __name__ == "__main__":
    print(poem_predict())

請輸入文字:黃金海岸的岸邊 我們肩並著肩 潔淨的藍天 清澈的水面 吻成一條海平線 看你溫柔的雙眼 彈著吉他的弦 歌詞是諾言 旋律是依戀 唱出一首五月天  五月的天 剛誕生的夏天 我們之間 才完成的愛戀 緊握的手裡面 有好多明天 五月的天 夢開始要鮮豔 前方蜿蜒 一長串的心願 我們一天一天 慢慢實現  大雨沖走了昨天 青春烏雲幾片 彩虹的旁邊 有星星幾點 迫不及待在眨眼 海風味道變香甜 沙灘鑲滿亮片 你哼著永遠 我和著不變,合唱一首五月天
白居易


In [9]:
if __name__ == "__main__":
    print(poem_predict_all())

請輸入文字：黃金海岸的岸邊 我們肩並著肩 潔淨的藍天 清澈的水面 吻成一條海平線 看你溫柔的雙眼 彈著吉他的弦 歌詞是諾言 旋律是依戀 唱出一首五月天  五月的天 剛誕生的夏天 我們之間 才完成的愛戀 緊握的手裡面 有好多明天 五月的天 夢開始要鮮豔 前方蜿蜒 一長串的心願 我們一天一天 慢慢實現  大雨沖走了昨天 青春烏雲幾片 彩虹的旁邊 有星星幾點 迫不及待在眨眼 海風味道變香甜 沙灘鑲滿亮片 你哼著永遠 我和著不變,合唱一首五月天
['李白機率： 0.05', '杜甫機率： 0.00', '白居易機率： 0.95']
